In [ ]:
!pip install geopandas

In [7]:
!hdfs dfs -mkdir /raw

In [2]:
!hadoop fs -rm -r /raw/data_raw.parquet

Deleted /raw/data_raw.parquet


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
import json

spark = SparkSession \
  .builder \
  .appName("streaming") \
  .master("local[*]") \
  .getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
schema = ArrayType(StructType([
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("date", StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("employee_id", IntegerType(), True),
    StructField("quantity_products", IntegerType(), True),
    StructField("order_id", StringType(), True)
]))

static_schema = StructType([
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("date", StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("employee_id", IntegerType(), True),
    StructField("quantity_products", IntegerType(), True),
    StructField("order_id", StringType(), True)
])

In [4]:
static_df = spark.createDataFrame([], static_schema)
n = 1
while n<=30:
    streaming_df = spark.readStream.format("socket").option("host", "localhost").option("port", "8000").load()
    json_df = streaming_df.select(from_json(col("value"), schema).alias("data"))
    json_df = json_df.selectExpr("explode(data) as dict").select("dict.*")
    writing_df = json_df.writeStream.format("memory").queryName("socketData").outputMode("update").start()
    static_stream_df = spark.sql("SELECT * FROM socketData")
    static_df = static_df.union(static_stream_df)
    writing_df.awaitTermination(2)
    writing_df.stop()
    n += 1
    if static_df.count() > 20:
        static_df.write.mode('append').parquet('/raw/data_raw.parquet')
        static_df = spark.createDataFrame([], static_schema)
    time.sleep(5)

In [10]:
df = spark.read.parquet("/raw/data_raw.parquet")
print("Tabla estatica:", df.count())
df = df.orderBy(desc("order_id"))
df.show(truncate=False)

Tabla estatica: 195
+------------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|latitude          |longitude         |date               |customer_id|employee_id|quantity_products|order_id                            |
+------------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|6.276178706065439 |-75.57310242733278|17/06/2024 00:17:03|2364       |6337       |252              |d8b9b417-b098-4344-b137-362894e4dczy|
|6.206311814587399 |-75.58018935979655|17/06/2024 00:17:03|3121       |9435       |204              |d8b9b417-b098-4344-b137-362894e4dczx|
|6.258995545343936 |-75.69974538611105|17/06/2024 00:17:03|2795       |6659       |525              |d8b9b417-b098-4344-b137-362894e4dczw|
|6.279884384261213 |-75.69014757062227|17/06/2024 00:17:03|5024       |1737       |373              |d8b9b417-b098-4344-b137-362894e4dczv|
|6.2690